In [35]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np
import pandas as pd
import seaborn as sns
import simdna
from simdna import synthetic
import statsmodels.api as sm
import torch
from tqdm.auto import tqdm

from utils import one_hot_decode
from in_silico_mutagenesis import compute_summary_statistics, generate_wt_mut_batches, write_results
from pyx.one_hot import one_hot
from tf_coop_model import CountsRegressor, IterablePandasDataset
from tf_coop_model import anscombe_transform, run_one_epoch, spearman_rho, pearson_r
from tf_coop_simulation import background_frequency
from tf_coop_simulation import simulate_counts, simulate_oracle_predictions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
np.random.seed(42)

In [37]:
sequence_length = 100

test_data_fpath = '../../dat/sim/test_labels.csv'
raw_simulation_data_fpath = '../../dat/sim/test_sequences.simdata'

sequences_col = "sequences"
label_cols = ["labels_exp", "labels_out"]
batch_size = 1000
n_samples = 10

exposure_motif = "GATA_disc1"
outcome_motif = "TAL1_known1"

In [38]:
torch.set_grad_enabled(False)

# Load Test Data & Generate Predictions
Now we're going to load test data to get some basic metrics about how well our model performs.

In [42]:
test_df = pd.read_csv(test_data_fpath)
test_dataset = IterablePandasDataset(
    test_df, x_cols=sequences_col, y_cols=label_cols, x_transform=one_hot,
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, num_workers=0
)
len(test_df)

1

In [40]:
both_motifs_df = test_df[(test_df['has_exposure'] == 1) & (test_df['has_outcome'] == 1)]
exposure_motif_df = test_df[(test_df['has_exposure'] == 1) & (test_df['has_outcome'] == 0)]
outcome_motif_df = test_df[(test_df['has_exposure'] == 0) & (test_df['has_outcome'] == 1)]
neither_motif_df = test_df[
    (test_df['has_exposure'] == 0) & (test_df['has_outcome'] == 0)
]

In [41]:
len(both_motifs_df), len(exposure_motif_df), len(outcome_motif_df), len(neither_motif_df)

(1, 0, 0, 0)

In [18]:
both_motifs_dataset = IterablePandasDataset(
    both_motifs_df, x_cols=sequences_col, y_cols=label_cols, x_transform=one_hot,
)
exposure_motif_dataset = IterablePandasDataset(
    exposure_motif_df, x_cols=sequences_col, y_cols=label_cols, x_transform=one_hot,
)

In [19]:
def mutate(seqs):
    preds = {}
    all_muts = []
    for seq in tqdm(seqs):
        muts = generate_wt_mut_batches(seq, seq.shape[0] * seq.shape[1]).squeeze()
        muts = muts.transpose(0, 1, 2)
        all_muts.append(muts)
    return np.array(all_muts)

In [20]:
both_motifs_sample_seqs = [x for x, y in both_motifs_dataset]

In [21]:
exposure_motif_sample_seqs = [x for x, y in exposure_motif_dataset]

In [22]:
sample_seqs = np.array([seq for seq, label in both_motifs_dataset])
sample_labels = np.array([label for _, label in both_motifs_dataset])
sample_muts = mutate(sample_seqs)

In [23]:
np.random.seed(42)

def simulate_adjusted_oracle_predictions(sequences, exposure_pwm, outcome_pwm, alpha=100, beta=100, perturb_by=1):
    q_exp, q_out = simulate_counts(sequences, exposure_pwm, outcome_pwm)
    q_exp = np.max((np.zeros_like(q_exp), q_exp-perturb_by), axis=0) # Different / impt line in this function.
    c_exp = alpha * q_exp
    c_out = beta * (q_exp * q_out)
    assert (np.isfinite(q_exp) & (q_exp >= 0)).all(), np.argwhere((np.isnan(q_exp) | (q_exp < 0)))
    c_exp_noisy = np.random.poisson(lam=c_exp, size=len(c_exp))
    c_out_noisy = np.random.poisson(lam=c_out, size=len(c_out))
    return c_exp_noisy, c_out_noisy

In [24]:
motifs = synthetic.LoadedEncodeMotifs(
    simdna.ENCODE_MOTIFS_PATH, pseudocountProb=0.001
)
exposure_pwm = motifs.loadedMotifs[exposure_motif].getRows()
outcome_pwm = motifs.loadedMotifs[outcome_motif].getRows()

In [25]:
# In-silico mut way of generating estimates
original_labels_scm = simulate_adjusted_oracle_predictions(
    [one_hot_decode(seq) for seq in sample_seqs],
    exposure_pwm,
    outcome_pwm,
    perturb_by=0
)
adjusted_labels_scm = simulate_adjusted_oracle_predictions(
    [one_hot_decode(seq) for seq in sample_seqs],
    exposure_pwm,
    outcome_pwm,
    perturb_by=.75
)
original_labels_scm = np.array(original_labels_scm)
adjusted_labels_scm = np.array(adjusted_labels_scm)
original_labels_scm, adjusted_labels_scm

(array([[ 96],
        [107]]),
 array([[19],
        [26]]))

In [26]:
original_labels_scm.shape, adjusted_labels_scm.shape

((2, 1), (2, 1))

In [27]:
scm_cis = ((adjusted_labels_scm[1, :] - original_labels_scm[1, :]) / 
           (adjusted_labels_scm[0, :] - original_labels_scm[0, :]))

In [28]:
original_labels_scm_anscombe = anscombe_transform(original_labels_scm)
adjusted_labels_scm_anscombe = anscombe_transform(adjusted_labels_scm)

In [29]:
scm_cis_anscombe = (
    (adjusted_labels_scm_anscombe[1, :] - original_labels_scm_anscombe[1, :]) / 
    (adjusted_labels_scm_anscombe[0, :] - original_labels_scm_anscombe[0, :])
)

In [30]:
adjusted_labels_ism = []
for i, muts in enumerate(tqdm(sample_muts)):
    adjusted_labels_ = simulate_oracle_predictions(
        [one_hot_decode(mut) for mut in muts],
        exposure_pwm,
        outcome_pwm
    )
    adjusted_labels_ism.append(adjusted_labels_)
adjusted_labels_ism = np.array(adjusted_labels_ism).reshape(len(sample_seqs), 4, 100, -1)
adjusted_labels_ism_anscombe = anscombe_transform(adjusted_labels_ism)

In [31]:
seq_idxs = np.array(sample_seqs).astype(np.bool)
adjusted_ref_labels_ism = adjusted_labels_ism_anscombe[seq_idxs].reshape(len(sample_seqs), 1, 100, -1)
adjusted_mut_labels_ism = adjusted_labels_ism_anscombe[~seq_idxs].reshape(len(sample_seqs), 3, 100, -1)
adjusted_diffs = adjusted_mut_labels_ism - adjusted_ref_labels_ism

In [32]:
ism_cis = []
for i in range(len(sample_seqs)):
    x = sm.add_constant(adjusted_diffs[i, :, :, 0].flatten(), prepend=False)
    y = adjusted_diffs[i, :, :, 1].flatten()
    ols_res = sm.OLS(y, x).fit()
    ism_cis.append(ols_res.params[0])

In [34]:
len(sample_seqs)

1

In [ ]:
ism_cis[:10]

In [ ]:
sns.distplot(ism_cis)